# Memory and Context in AI Agents


## Memory
Without memory, AI agents are essentially stateless. They forget everything after each request, leading to a lack of continuity in conversation or execution.  
Imagine telling an AI your name, and just moments later, it doesn't remember it.



Without Memory:  
- *User: "Remember my name is John."*  
- *AI: "Okay!"*  
- *User: "What's my name?"*  
- *AI: "I don’t know."*  

With Memory:  
- *AI: "Your name is John."*   


## Context
Context is crucial for seamless interactions and ensures that conversations with the AI feel natural and coherent.  
It also allows the AI to handle multi-step tasks effectively, enhancing the overall user experience


Without Context:  
- *User: "Book a flight to NYC."*    
- *User: "What's the weather?"*  
- *AI: "I don't understand the relation."*  

With Context:  
- *AI knows user wants weather information for NYC*



First let's setup our configuration and create our basic agent

## Configuration

Like in the previous exercise we need to setup LLM and configure keys and endpoints

In [1]:
var Endpoint = "<REPLACE_WITH_AZURE_OPENAI_ENDPOINT>";
var Key = "<REPLACE_WITH_AZURE_OPENAI_API_KEY>";
var ChatModelId = "gpt-4o"; // use any model you deployed
var EmbeddingModelId = "text-embedding-3-small";

## Creating Agent Without Context

In [2]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3"
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.4 Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3

In [3]:
using System.ComponentModel;
using System.ClientModel;
using Microsoft.Extensions.AI;
using Azure.AI.OpenAI;

In [4]:
var client = new AzureOpenAIClient(new Uri(Endpoint), new ApiKeyCredential(Key))
        .AsChatClient(ChatModelId)
        .AsBuilder()
        .UseFunctionInvocation() // allows to call registered functions
        .Build();

### Creating Tools

Let's create 3 tools
- Getting the weather information
- Booking a flight
- Getting travelling advice

In [8]:
[Description("Gets the weather")]
async Task<string> GetWeather(string destination)
{
    return $"It's {(Random.Shared.NextDouble() > 0.5 ? "sunny" : "raining")} in {destination}";
}

[Description("Books a flight to a given destination")]
async Task<string> BookFlight(string userRequest, string destination)
{
    return $"Your flight to {destination} is booked.";
}

[Description("Gets travel-related advice")]
async Task<string> GetTravelAdvice(string userRequest, string destination)
{
    return destination;
}

In [10]:
var chatOptions = new ChatOptions
{
    Tools = 
    [
        AIFunctionFactory.Create((string destination) => GetWeather(destination)),
        AIFunctionFactory.Create(BookFlight),
        AIFunctionFactory.Create(GetTravelAdvice)
    ],
    ToolMode = ChatToolMode.RequireAny
};

Now let's ask our agent 2 things:
- Book a flight to NY
- Give us vacation clothes recommendation

In [11]:
await foreach (var message in client.GetStreamingResponseAsync("Book a flight to NY.", chatOptions))
{
    Console.Write(message);
}
Console.WriteLine();

await foreach (var message in client.GetStreamingResponseAsync("What should I wear on my vacation?", chatOptions))
{
    Console.Write(message);
}

Your flight to New York (NY) is booked!
Could you please tell me the destination for your vacation? This will help me provide better advice on what to wear.

As expected agent cant make an observation that both questions are related. Because there is no context
>Your flight to NY is booked.  
I need to know your vacation destination to provide advice about what you should wear. Could you let me know where you're going?

So let's add one.

## Adding Context

Luckily for us memory and context could be added in a similar fashion - through a short term or a long term memory  
In our example we will be using vector db [Chroma](https://www.trychroma.com) which we can use with their SDK  
We can run it locally in a [docker container](https://hub.docker.com/r/chromadb/chroma)  

In [12]:
#r "nuget: ChromaDB.Client, 1.0.1-ci-13369893450"

Installed Packages ChromaDB.Client, 1.0.1-ci-13369893450

### Chroma Client

Let's create a ChromaCollectionClient using our hosted instance of Chroma.  
We will also create a collection for our embeddings "chat_memory"

In [13]:
using ChromaDB.Client;
using System.Net.Http;

public static async Task<ChromaCollectionClient> GetChromaCollectionClient()
{
    var configOptions = new ChromaConfigurationOptions(uri: "http://localhost:8000/api/v1/");
    var httpClient = new HttpClient();
    var chromaClient = new ChromaClient(configOptions, httpClient);
    var collection = await chromaClient.GetOrCreateCollection("chat_memory");
    return new ChromaCollectionClient(collection, configOptions, httpClient);
}


### Embedding Generator

Embeddig generator will translate our text into embeddigs (vectors)  
Using Microsoft.Extensions.AI will allow us to do it easily

In [14]:
var generator =  new AzureOpenAIClient(
                new Uri(Endpoint),
                new ApiKeyCredential(Key))
            .AsEmbeddingGenerator(EmbeddingModelId);

Now let's create 2 helper methods
- Storing flight booking information
- Fetching relevant advice


In [15]:
private async Task<string> StoreFlightBooking(string userRequest, string destination)
{
    var response = $"Your flight to {destination} is booked.";

    var collectionClient = await GetChromaCollectionClient();
    var embedding = await generator.GenerateEmbeddingVectorAsync(userRequest);

    await collectionClient.Add(
        [Guid.NewGuid().ToString()],
        [embedding],
        [
            new Dictionary<string, object>
            {
                ["Type"] = "FlightBooking",
                ["Destination"] = destination,
                ["UserRequest"] = userRequest // Store the original request as metadata
            }
        ]
    );

    return response;
}

In [16]:
private async Task<string> FetchRelevantAdvice(string userRequest)
{
    var collectionClient = await GetChromaCollectionClient();
    var embedding = await generator.GenerateEmbeddingVectorAsync(userRequest);

    // Query ChromaDB using embedding, requesting the top 1 result based on similarity
    var searchResults = await collectionClient.Query(embedding , 1, include: ChromaQueryInclude.Distances | ChromaQueryInclude.Metadatas);

    if (searchResults.Any())
    {
        var metadata = searchResults.First().Metadata;
        var destination = metadata!["Destination"].ToString();
        var userRequestContext = metadata["UserRequest"].ToString();

        // Leaving it up to the AI to call the `GetWeather` function using description
        return $"Based on your trip to {destination} and your previous request '{userRequestContext}', let me get the current weather for you.";
    }
    else
    {
        return "I'm sorry, I couldn't find any related travel advice for your request.";
    }
}

Let's update our methods to use this context

In [17]:
[Description("Books a flight to a given destination")]
async Task<string> BookFlight(string userRequest, string destination)
{
    var response = await StoreFlightBooking(userRequest, destination);
    return response;
}

[Description("Gets travel-related advice")]
async Task<string> GetTravelAdvice(string userRequest)
{
    var response = await FetchRelevantAdvice(userRequest);
    return response;
}

Now let's ask our agent the same questions once again and see the result

In [ ]:
await foreach (var message in client.GetStreamingResponseAsync("Book a flight to NY.", chatOptions))
{
    Console.Write(message);
}
Console.WriteLine();

await foreach (var message in client.GetStreamingResponseAsync("What should I wear on my vacation?", chatOptions))
{
    Console.Write(message);
}

>Your flight to New York (NY) is booked! Safe travels!  
Since it's sunny in New York, I recommend wearing light and breathable clothing. Here are some suggestions:
>- **T-shirt or tank top**: Choose lightweight fabrics.
>- **Shorts or lightweight pants**: Comfortable options for walking around.
>- **Sunglasses and a hat**: To protect yourself from the sun.
>- **Sandals or comfortable walking shoes**: Perfect for exploring the city.

>Don't forget to apply sunscreen! Enjoy your vacation!


As you can see now with the context it's able to recognize that our question about vacation is probably related to our previous request to book a flight to NY

### Final Thoughts

We learnt why memory and context are very important concepts of AI agents  
and we also learnt how to handle it using Microsoft.Extensions.AI and Chroma vector db